## Preprocess video 과정  
1. **환경 세팅**  
- 작업 디렉토리(HOME) 및 장치(CPU/GPU) 설정
- 필수 라이브러리(SAM2, YOLOv11) 설치
- SAM2 체크포인트 및 YOLOv11 학습 모델 다운로드

2. **sam2 모델 초기화**
- CONFIG 및 CHECKPOINT 로드
- SAM2 모델 불러오기 (build_sam2_video_predictor)

3. **비디오 정보 확인**  
- fps, 총 프레임 수, 비디오 길이 출력  

4. **비디오 -> 프레임 분할 및 저장**  
- 비디오를 프레임 이미지(JPEG)로 분리
- 프레임 크기 50% 축소 후 디스크에 저장
- 전처리된 프레임들을 ZIP 파일로 압축 및 다운로드  

5. **sam2 초기화**  
- 프레임들을 불러와 inference 상태 초기화
- 기존 상태가 있으면 reset 후 초기화

### 1. 환경 세팅  

In [ ]:
# NOTE: To make it easier for us to manage datasets, images and models we create a HOME constant.
import os

HOME = os.getcwd()
print("HOME:", HOME)

In [ ]:
# Install SAM2 and dependencies
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd {HOME}/segment-anything-2
!pip install -e . -q
!python setup.py build_ext --inplace

# 필수 라이브러리 설치
!pip install -q supervision[assets] jupyter_bbox_widget
!pip install ultralytics

In [ ]:
# Download SAM2 checkpoints
!mkdir -p {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P {HOME}/checkpoints

# yolo11s best model
!gdown 1A6_iThtWBlzgvGqOP9QXuDYGFL809jYL

In [ ]:
# 라이브러리 imports
import cv2
import torch
import base64
import numpy as np
import supervision as sv

from pathlib import Path
from supervision.assets import download_assets, VideoAssets
from sam2.build_sam import build_sam2_video_predictor

import shutil
from google.colab import files

IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

### 2. 모델 초기화  

In [ ]:
# NOTE: This code enables mixed-precision computing for faster deep learning. It uses bfloat16 for most calculations and, on newer NVIDIA GPUs, leverages TensorFloat-32 (TF32) for certain operations to further boost performance.
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = f"{HOME}/checkpoints/sam2_hiera_large.pt"
CONFIG = "sam2_hiera_l.yaml"

# sam2 model 초기화
sam2_model = build_sam2_video_predictor(CONFIG, CHECKPOINT)  # SAM2 모델 빌드 및 가중치로드

### 3. 비디오 정보 확인
- fps, 프레임 수, 비디오 길이 출력

In [ ]:
video_path = "/content/test video 07.mp4" # 사용할 비디오 경로

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 원본 총 프레임 개수
duration = frame_count / fps  # 계산된 비디오 길이
cap.release()

print(f"비디오 FPS: {fps:.6f}")
print(f"비디오 총 프레임 개수: {frame_count}")
print(f"계산된 비디오 길이: {duration:.2f}초")

### 4. 비디오 -> 프레임 분할 및 저장  
- sv.get_video_frames_generator(SOURCE_VIDEO, start=0, end=None) 함수를 사용  
- 1초에 몇 개의 프레임이 저장되었는지 확인 가능  

In [ ]:
SCALE_FACTOR = 0.5  # 이미지 크기 50% 축소
SOURCE_VIDEO = "/content/test video 07.mp4"
SOURCE_FRAMES = Path(HOME) / Path(SOURCE_VIDEO).stem
SOURCE_FRAMES.mkdir(parents=True, exist_ok=True)

frames_generator = sv.get_video_frames_generator(SOURCE_VIDEO, start=0, end=None)

images_sink = sv.ImageSink(
    target_dir_path=SOURCE_FRAMES.as_posix(),
    overwrite=True,
    image_name_pattern="{:05d}.jpg"
)

with images_sink:
    for frame in frames_generator:
        frame = sv.scale_image(frame, SCALE_FACTOR)
        images_sink.save_image(frame)

# 첫번째 프레임 및 결과 및 저장 경로
SOURCE_IMAGE = SOURCE_FRAMES / "00000.jpg"
TARGET_VIDEO = Path(HOME) / f"{Path(SOURCE_VIDEO).stem}-result-07.mp4"

In [ ]:
# 프레임 압축 및 다운로드
folder_path = "/content/test video 07"          # 저장된 프레임이 있는 폴더
zip_path = "/content/test_video_frames_07.zip"  # 압축 파일명

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)
files.download(zip_path)

### 5. SAM2 모델 초기화 (inference 대비용)  
- NOTE: 만약 inference_state를 사용하여 이전에 **트래킹(Tracking, 추적)**을 실행했다면, 먼저 reset_state()를 호출하여 초기화해야 함  

In [ ]:
# sam2 모델 초기화
inference_state = sam2_model.init_state(video_path=SOURCE_FRAMES.as_posix())
sam2_model.reset_state(inference_state)